In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.patches import Circle
from sklearn import metrics
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
import sys
import cv2
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import os
from torchvision.io import read_image

# Load dataset

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
            
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #image = torch.from_numpy(np.load(img_path + ".npy").reshape(3,224,224))
        image = read_image(img_path)
        label = torch.Tensor([self.img_labels.iloc[idx, 1].astype(np.float32)])
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

num_epochs = 50
batch_size = 32
learning_rate = 0.0001       
    
dataset = CustomImageDataset('data - Copy.csv', 'lineImages/')    
    
dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create model

In [3]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,9,(3,3), padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(9,18,(3,3), padding=1)
        self.conv3 = nn.Conv2d(18,36,(3,3), padding=1)
        self.conv4 = nn.Conv2d(36,64,(3,3), padding=1)
        self.conv5 = nn.Conv2d(64,128,(3,3), padding=1)
        self.fc1 = nn.Linear(100352,256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x, test):
#         if test == False:
#             plt.imshow(x[0].permute(1,2,0))
#             plt.show()
#         else:
#             plt.imshow(x.permute(1,2,0))
#             plt.show()
            
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        
        if test == False:
            x = torch.flatten(x,1) # flatten all dimensions except batch
        else:
            x = torch.flatten(x)
        #x = torch.flatten(x)    
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x


net = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [4]:
def scaleImage(img):
    q = img.numpy()
    q = q/255
    q = torch.from_numpy(q)
    return q

# Train model

In [5]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (images,labels) in enumerate(dataset):
        
        optimizer.zero_grad()
        images = scaleImage(images)
        #labels = scaleLabel(labels)
        
        images = images.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        
        # forward + backward + optimize
        
        
        outputs = net(images.float(), False)
        
        labels = labels.unsqueeze(1)
        #print(str(labels) + " " + str(outputs))
        loss = criterion(outputs.unsqueeze(1), labels)
        #print(loss)
        loss.backward()
        optimizer.step()

        
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 10 mini-batches
            print(f'[Epoch: {epoch + 1}, Tuple: {i + 1:5d}] loss: {running_loss / 9:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch: 1, Tuple:     1] loss: 0.016
[Epoch: 2, Tuple:     1] loss: 0.014
[Epoch: 3, Tuple:     1] loss: 0.007
[Epoch: 4, Tuple:     1] loss: 0.021
[Epoch: 5, Tuple:     1] loss: 0.014
[Epoch: 6, Tuple:     1] loss: 0.014
[Epoch: 7, Tuple:     1] loss: 0.019
[Epoch: 8, Tuple:     1] loss: 0.008
[Epoch: 9, Tuple:     1] loss: 0.010
[Epoch: 10, Tuple:     1] loss: 0.011
[Epoch: 11, Tuple:     1] loss: 0.008
[Epoch: 12, Tuple:     1] loss: 0.014
[Epoch: 13, Tuple:     1] loss: 0.008
[Epoch: 14, Tuple:     1] loss: 0.012
[Epoch: 15, Tuple:     1] loss: 0.008
[Epoch: 16, Tuple:     1] loss: 0.009
[Epoch: 17, Tuple:     1] loss: 0.007
[Epoch: 18, Tuple:     1] loss: 0.009
[Epoch: 19, Tuple:     1] loss: 0.005
[Epoch: 20, Tuple:     1] loss: 0.003
[Epoch: 21, Tuple:     1] loss: 0.002
[Epoch: 22, Tuple:     1] loss: 0.002
[Epoch: 23, Tuple:     1] loss: 0.001
[Epoch: 24, Tuple:     1] loss: 0.004
[Epoch: 25, Tuple:     1] loss: 0.002
[Epoch: 26, Tuple:     1] loss: 0.003
[Epoch: 27, Tuple:   

In [6]:
PATH = './jetson.pth'
torch.save(net.state_dict(), PATH)

# Loading the trained network
#net.load_state_dict(torch.load(PATH))

In [7]:
correct = 0
total = 0
print("Actual: Prediction:")
dataset = CustomImageDataset('data - Copy.csv', 'lineImages/')    
#net.eval()
with torch.no_grad():
    for data in dataset:
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        
        images = scaleImage(images)
        outputs = net(images.float(), True)
        predicted = outputs.data
        
        total += labels.size(0)
        correct += (outputs.data == labels).sum().item()
        print(str(np.asarray(labels)) + ", " + str(np.asarray(outputs.data)))

Actual: Prediction:
[0.], [-0.01855229]
[0.], [-0.02725376]
[0.], [-0.09067733]
[0.], [-0.00132646]
[0.5], [0.64229125]
[-0.25], [-0.23426959]
[-0.25], [-0.27955362]
[-0.25], [-0.3192867]
[0.], [-0.04313525]
[0.], [-0.04068213]
[0.1], [0.20859954]
[0.], [-0.01031652]
[0.05], [0.04648495]
[-0.1], [-0.1177081]
[-0.5], [-0.51397645]
[-0.05], [-0.14761779]
[0.5], [0.58812016]
[0.25], [0.27976763]
[-0.1], [-0.16445331]
[-0.05], [-0.10444861]
[0.], [-0.00330352]
[0.025], [0.0927512]
[-0.5], [-0.7433291]
[0.1], [0.06320482]
[-0.05], [-0.07102908]
[-0.7], [-0.7018535]
[0.], [-0.00063344]
[0.], [0.02441997]
[0.5], [0.5602082]
[1.], [0.9550242]
[1.], [0.8151367]
[1.], [0.9428721]
[0.5], [0.61454546]
[-1.], [-0.87560445]
[0.025], [0.04964324]
[-0.25], [-0.22147986]
[-0.3], [-0.41009998]
[0.05], [0.09789502]
[-0.5], [-0.6282585]
[0.05], [0.04076931]
[0.05], [0.13907929]
[0.025], [0.01123299]
[0.2], [0.22114508]
[0.], [-0.00284143]
[0.7], [0.6779291]
[-0.5], [-0.50869733]
[0.], [-0.02458373]
[-0.02